<a href="https://colab.research.google.com/github/nihald2000/Edgeai/blob/main/RAGChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

In [3]:
!pip install -q \
 langchain_community \
 langchain_groq \
 langchain_community

In [4]:
import os
from google.colab import userdata
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
if GROQ_API_KEY:
    os.environ["GROQ_API_KEY"] = GROQ_API_KEY
    print(" GROQ_API_KEY loaded.")
else:
    print("GROQ_API_KEY not found in Colab Secrets. LangSmith tracing will not work.")

 GROQ_API_KEY loaded.


In [6]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=GROQ_API_KEY)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x79e19de086d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x79e19d663cd0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Nihal")])

AIMessage(content="Hello Nihal, it's nice to meet you! 👋 \n\nIs there anything I can help you with today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 16, 'total_tokens': 44, 'completion_time': 0.050909091, 'prompt_time': 0.001910831, 'queue_time': 0.091344001, 'total_time': 0.052819922}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--c30b6250-2536-408c-a26e-4db7866e875e-0', usage_metadata={'input_tokens': 16, 'output_tokens': 28, 'total_tokens': 44})

In [8]:

from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Nihal and I am a Chief AI Engineer"),
        AIMessage(content="Hello Nihal! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="According to our conversation, your name is Nihal and you are a Chief AI Engineer.  😊 \n\nIs there anything else you'd like to tell me about yourself or your work? I'm happy to chat! \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 99, 'total_tokens': 149, 'completion_time': 0.090909091, 'prompt_time': 0.004274549, 'queue_time': 0.091769801, 'total_time': 0.09518364}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--cf689ade-5b8a-47bb-8ade-3be210517c9b-0', usage_metadata={'input_tokens': 99, 'output_tokens': 50, 'total_tokens': 149})

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [10]:
config={"configurable":{"session_id":"chat1"}}

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Nihal")],
    config=config
)

In [12]:
response.content

"Hello Nihal, it's nice to meet you! 👋 \n\nIs there anything I can help you with today? 😊\n\n"

In [13]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Nihal.  😊 \n\nI remembered from our earlier introduction!  \n\n\n\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 59, 'total_tokens': 81, 'completion_time': 0.04, 'prompt_time': 0.003396224, 'queue_time': 0.09179393799999999, 'total_time': 0.043396224}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--68c7d011-a078-4c8d-b29e-b458f2d2d5ea-0', usage_metadata={'input_tokens': 59, 'output_tokens': 22, 'total_tokens': 81})

In [14]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name.\n\nIf you'd like to tell me your name, I'd be happy to use it! 😊\n"

In [15]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hello John! It's nice to meet you.  \n\nWhat can I do for you today? 😊  \n"

In [16]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John, remember?  😊 \n\nIs there anything else I can help you with?\n'

In [17]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [22]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Nihal")]})

AIMessage(content="Hi Nihal, it's nice to meet you!  I'm happy to help with any questions you have.  \n\nWhat can I do for you today? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 32, 'total_tokens': 72, 'completion_time': 0.072727273, 'prompt_time': 0.003436464, 'queue_time': 0.091676529, 'total_time': 0.076163737}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--b34cf881-c49b-43b6-b2e9-fb34ace43085-0', usage_metadata={'input_tokens': 32, 'output_tokens': 40, 'total_tokens': 72})

In [23]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [24]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Nihal")],
    config=config
)

response

AIMessage(content="Hi Nihal!  \n\nIt's great to meet you.  \n\nIs there anything I can help you with today?  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 115, 'total_tokens': 146, 'completion_time': 0.056363636, 'prompt_time': 0.005961582, 'queue_time': 0.08481593100000001, 'total_time': 0.062325218}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--a458c18c-68d4-4835-b9a2-2a1e37fa8598-0', usage_metadata={'input_tokens': 115, 'output_tokens': 31, 'total_tokens': 146})

In [25]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"Your name is Nihal.  👋 \n\nIt's nice to have you here!  \n\nLet me know if you have any questions or tasks for me.\n"

In [26]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [27]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Nihal")],"language":"Hindi"})
response.content

'नमस्ते कृष्ण! 👋\n\nमुझे बहुत खुशी है कि तुम मेरे साथ बात करना चाहते हो।  \n\nआप मुझे क्या पूछना चाहेंगे? मैं आपकी हर मदद करने के लिए तैयार हूँ! 😄\n\n'

In [28]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [39]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Nihal")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते निहाल! 😊 \n\nमैं आपकी मदद करने के लिए तैयार हूँ। आप मुझसे कोई भी प्रश्न पूछ सकते हैं। \n'

In [40]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [41]:
response.content

'मुझे नहीं पता आपका नाम क्या है। क्या आप मुझे बता सकते हैं? 😊 \n'

In [42]:
### Managing the Conversation History


In [43]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [44]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model

)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As a helpful but imaginary assistant, I don't have access to your personal preferences like your favorite ice cream flavor. \n\nWhat's your favorite ice cream flavor?  🍦 😄  \n"

In [45]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked what 2 + 2 equals. 😊  \n\n\n\nLet me know if you want to try another one!\n'

In [46]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [47]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I have no memory of past conversations and don't know your name.  \n\nWould you like to tell me your name? 😊  \n"

In [48]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. So I don't know what math problem you asked.\n\nPlease ask me your math problem and I'll be happy to help! 😊  \n\n"